In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import molpot as mpot
import torch

In [3]:
qm9_dataset = mpot.dataset.QM9(
        save_dir="qm9", batch_size=3, total=1000, device="cpu"
    )
dp = qm9_dataset.prepare().calc_nblist(5).atomic_dress(types_list=[1, 6, 7, 8, 9], key=mpot.Alias.Z, prop=mpot.Alias.qm9.U0, buffer=1000).convert_unit(mpot.Alias.qm9.U0, 'kcal/N_A/mol').convert_unit(mpot.Alias.R, 'angstrom')

train, valid = dp.random_split(
    weights={"train": 0.8, "valid": 0.2}, seed=42
)

train_dataloader = mpot.create_dataloader(train)
valid_dataloader = mpot.create_dataloader(valid)

627.5094740630659
1


In [4]:
n_atom_basis = 10
arch = mpot.nnp.PaiNN(
    n_atom_basis, 4, mpot.nnp.GaussianRBF(n_atom_basis, 5), mpot.nnp.CosineCutoff(5)
)
readout = mpot.nnp.Atomwise(n_in = n_atom_basis, n_out=1, n_hidden=None, input_key=mpot.Alias.painn.scalar, output_key=mpot.Alias.energy, aggregation_mode='sum')
model = mpot.NNPotential("PaiNN", arch, readout, derive_energy=False)
criterion = mpot.MultiMSELoss([1], targets=[(mpot.Alias.energy, mpot.Alias.qm9.U0)])
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.9)

stagnation = mpot.strategy.Stagnation(mpot.Alias.loss, patience=torch.inf)

trainer = mpot.Trainer(
    "painn-qm9",
    model,
    criterion,
    optimizer,
    scheduler,
    train_dataloader,
    valid_dataloader,
    strategies=[stagnation],
    logger={
        "metrics": {
            "speed": mpot.metric.StepSpeed(),
            "loss": mpot.metric.Identity(mpot.Alias.loss),
            "energy_mae": mpot.metric.MAE(
                mpot.Alias.energy, mpot.Alias.qm9.U0
            ),
        },
        "handlers": [mpot.logger.ConsoleHandler(), mpot.logger.TensorBoardHandler()],
        "save_dir": "./log",
    },
    config={
        "save_dir": "model",
        "device": {"type": "cpu"},
        "compile": False,
        "report_rate": 10,
        "valid_rate": 10000,
        "modify_lr_rate": 1000,
        "checkpoint_rate": 10000,
    },
    train_hooks = [
        lambda nstep, model, outputs : torch.nn.utils.clip_grad_norm_(model.parameters(), 0.01),
        # lambda nstep, model, outputs : print(outputs[mpot.Alias.energy])
    ]
)


output = trainer.train(5)


step | epoch |  speed      | loss       | energy_mae


INFO:root:Reach the max step 5


0 | 0 | 2.7183     | 1.1109     | 1.0368    
